<a href="https://colab.research.google.com/github/jjmora22/NLP/blob/Codigo_Notebooks/4_Metricas_Conclusiones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Métricas y conclusiones sobre el modelo Random Forest

Éstas son las librerías que utilizaré

In [31]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline # Enlaza diferentes pasos en un "workflow"
from imblearn.over_sampling import RandomOverSampler # Voy a balancear la muestra, porque resultó muy pequeña para la validación
import pickle # Permite guardar/recuperar del disco el modelo Random Forest que vamos a utilizar
import joblib # cargar el modelo entrenado a memoria 
from joblib import load
from numpy import loadtxt # para cargar el numpy.array de las proyecciones de validación

In [2]:
!pip install --upgrade scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install scikit-learn-intelex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
!pip uninstall scikit-learn -y

!pip install -U scikit-learn

Found existing installation: scikit-learn 1.0.2
Uninstalling scikit-learn-1.0.2:
  Successfully uninstalled scikit-learn-1.0.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 40.0 MB/s eta 0:00:00


In [4]:
from sklearnex import patch_sklearn # Permite el cómputo distribuido
patch_sklearn() # Corrige errores en joblib (scikit-learn)


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


Primero recupero el dataframe y el modelo entrenado --> 

In [26]:
# Carga los datos de validación
X_val = pd.read_csv('X_val.csv')
y_val = pd.read_csv('y_val.csv', header='infer')
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')

#oversampler = RandomOverSampler()
#X_val, y_val = oversampler.fit_resample(X_train, y_train)

# Carga el modelo random forest y el vectorizador pre-entrenado
rf_model = joblib.load('random_forest_model.joblib')
vectorizer = joblib.load('vectorizer.pkl')

# Hago fit de X_val para el vectorizador 
#vectorizer = TfidfVectorizer(max_df=1, ngram_range=(1, 2))
#X_train_vectorized = vectorizer.fit_transform(X_train)

# Transforma los datos de validación usando el vectorizador cargado
X_val_vectorized = vectorizer.transform(X_val)

# Realiza predicciones en los datos transformados usando el modelo cargado
y_pred = loadtxt('rfc_val_pred.csv', delimiter=',')

In [18]:
X_val_vectorized.shape

(1, 117355)

In [23]:
#y_val.shape
#y_pred.shape
#X_val.shape
#X_train.shape
#X_val_vectorized.shape
print(y_val.value_counts())

overall
5          947
1          550
4          331
3          227
2          186
dtype: int64


Tomando en cuenta los resultados del modelo escogido, calcularé las métricas específicas que validen la bondad del modelo, utilizo los datos de validación. 

In [27]:
# Calcula la precisión (accuracy) del modelo
precision = accuracy_score(y_val, y_pred)
print(f"Precisión: {precision:.4f}")

# Imprime la matriz de confusión y el reporte de clasificación
cm = confusion_matrix(y_val, y_pred)
print("Matriz de confusión:")
print(cm)
cr = classification_report(y_val, y_pred)
print("Reporte de clasificación:")
print(cr)

Precisión: 0.5872
Matriz de confusión:
[[413   6   2   0 129]
 [104   2   4   3  73]
 [ 92   0   6   6 123]
 [ 44   2   0  15 270]
 [ 61   2   1   3 880]]
Reporte de clasificación:
              precision    recall  f1-score   support

           1       0.58      0.75      0.65       550
           2       0.17      0.01      0.02       186
           3       0.46      0.03      0.05       227
           4       0.56      0.05      0.08       331
           5       0.60      0.93      0.73       947

    accuracy                           0.59      2241
   macro avg       0.47      0.35      0.31      2241
weighted avg       0.54      0.59      0.49      2241



Se confirma el nivel de accuracy encontrado en train y test. Aunque el modelo es muy mejorable, se comporta de manera consistente con lo hallado. 

##Conclusiones FINALES



1. De un análisis exploratorio, se intuye que la satisfacción de los clientes está relacionada con el funcionamiento del producto/servicio y con su facilidad de uso. En los n-gramas fue claro tanto en el análisis de calificaciones postivas como en negativas la presencia de palabras relacionadas con estos dos conceptos.  
2. Se detectó la presencia de sugerencias de mejora de manera importante. Esto genera una fuente de información importante para los vendedores de Amazon. 
3. He elegido Naive Bayes Multinomial y Random Forest Classifier. 
4. Elegí Naive Bayes pues asume que la aparición de una palabra es independiente de la ocurrencia de otras palabras. Esto simplifica el algoritmo y lo hace más eficiente. En el caso de esta práctica, etiquetamos los datos con un sentimiento positivo o negativo y entrenamos al modelo para predecir el sentimiento (estrellas) de reviews futuras.  
5. En el caso de Random Forest Classifier, busca predecir el sentimiento, en base a agrupar elementos etiquetados de manera aleatoria y cada árbol se entrena sobre estos elementos. El resultado se obtiene de la predicción conjunta de varios árboles (el bosque). Permite el manejo de datasets grandes con alta dimensionalidad (gran cantidad de palabras), por lo que puede favorecer el overfitting. Random Forest limitar este riesgo al crear bosques de manera random. 
6. Estos modelos brindan una alta precisión en tareas de clasificación de texto. También son una buena alternativa en situaciones de alta dimensionalidad. 
7. Naive Bayes permite trabajar ambientes donde se encuentra información con "ruido". En nuestro caso, texto escrito por los usuarios. 
8. Ambos modelos son relativamente rápidos para entrenarse y hacer predicciones (aunque en las pruebas Random Forest ha tomado 2 horas vs Naive Bayes que lo resuelve en segundos).
9. Al evaluar los resultados del modelo, se observa que el método de Random Forest Classifier supera al de Naive Bayes en términos de Recall, lo que indica que el modelo de Random Forest tiene un mejor desempeño en identificar los verdaderos positivos, es decir, los casos en que el modelo predijo correctamente que una reseña era positiva o negativa.
10. En términos de precisión, los modelos obtienen puntajes similares. Esto indica que ambos modelos logran identificar correctamente un porcentaje aceptable de las reseñas, pero no son perfectos y pueden cometer algunos errores.
11. Es importante destacar que se puede seguir mejorando la precisión y el rendimiento del modelo mediante la optimización de los hiperparámetros o la incorporación de otras técnicas de preprocesamiento.
12. El análisis exploratorio y la implementación de modelos de clasificación de texto en esta práctica demuestran la utilidad de las técnicas de procesamiento de lenguaje natural para analizar grandes cantidades de datos no estructurados y obtener información valiosa para la toma de decisiones en el ámbito empresarial.


